In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf

In [73]:
df = pd.read_csv('train.csv')
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2000 non-null   int64  
 1   blue           2000 non-null   int64  
 2   clock_speed    2000 non-null   float64
 3   dual_sim       2000 non-null   int64  
 4   fc             2000 non-null   int64  
 5   four_g         2000 non-null   int64  
 6   int_memory     2000 non-null   int64  
 7   m_dep          2000 non-null   float64
 8   mobile_wt      2000 non-null   int64  
 9   n_cores        2000 non-null   int64  
 10  pc             2000 non-null   int64  
 11  px_height      2000 non-null   int64  
 12  px_width       2000 non-null   int64  
 13  ram            2000 non-null   int64  
 14  sc_h           2000 non-null   int64  
 15  sc_w           2000 non-null   int64  
 16  talk_time      2000 non-null   int64  
 17  three_g        2000 non-null   int64  
 18  touch_sc

In [75]:
df.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [76]:
df.duplicated().sum()

0

In [77]:
df['price_range'].value_counts()

1    500
2    500
3    500
0    500
Name: price_range, dtype: int64

In [78]:
x = df.drop('price_range',axis=1)
y = df['price_range']

In [79]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,random_state=1,test_size=0.3)

In [80]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
xtrain = ss.fit_transform(xtrain)
xtest = ss.transform(xtest)

In [81]:
xtrain.shape

(1400, 20)

In [82]:
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dropout

model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=24,activation='relu',kernel_regularizer=regularizers.l2(0.01),input_shape=(20,)),Dropout(0.50),
    tf.keras.layers.Dense(units=4,activation='softmax',kernel_regularizer=regularizers.l2(0.01))
])

In [83]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 24)                504       
                                                                 
 dropout_1 (Dropout)         (None, 24)                0         
                                                                 
 dense_7 (Dense)             (None, 4)                 100       
                                                                 
Total params: 604
Trainable params: 604
Non-trainable params: 0
_________________________________________________________________


In [84]:
model.compile(optimizer='sgd',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [85]:
from tensorflow.keras.callbacks import EarlyStopping

cb = EarlyStopping(
    monitor = 'val_loss',
    min_delta = 0.00001,
    patience = 20,
    verbose = 1,
    mode = 'auto',
    baseline = None,
    restore_best_weights = False
)

In [86]:
trained_model = model.fit(xtrain,ytrain,epochs=5000,callbacks=cb,validation_data=(xtest,ytest))

Epoch 1/5000
44/44 [==============================] - 1s 8ms/step - loss: 1.9005 - accuracy: 0.2829 - val_loss: 1.6329 - val_accuracy: 0.3250
Epoch 2/5000
44/44 [==============================] - 0s 4ms/step - loss: 1.7432 - accuracy: 0.3093 - val_loss: 1.5555 - val_accuracy: 0.3617
Epoch 3/5000
44/44 [==============================] - 0s 3ms/step - loss: 1.6656 - accuracy: 0.3114 - val_loss: 1.4966 - val_accuracy: 0.4083
Epoch 4/5000
44/44 [==============================] - 0s 3ms/step - loss: 1.5831 - accuracy: 0.3657 - val_loss: 1.4503 - val_accuracy: 0.4367
Epoch 5/5000
44/44 [==============================] - 0s 3ms/step - loss: 1.5040 - accuracy: 0.4207 - val_loss: 1.4097 - val_accuracy: 0.4750
Epoch 6/5000
44/44 [==============================] - 0s 4ms/step - loss: 1.4751 - accuracy: 0.3993 - val_loss: 1.3729 - val_accuracy: 0.4983
Epoch 7/5000
44/44 [==============================] - 0s 4ms/step - loss: 1.4348 - accuracy: 0.4379 - val_loss: 1.3392 - val_accuracy: 0.5233
Epoch 

In [87]:
print('Training_loss :',model.evaluate(xtrain,ytrain))
print('Testing_loss :',model.evaluate(xtest,ytest))

44/44 [==============================] - 0s 2ms/step - loss: 0.5306 - accuracy: 0.9714
Training_loss : [0.5305634140968323, 0.9714285731315613]
19/19 [==============================] - 0s 2ms/step - loss: 0.5601 - accuracy: 0.9517
Testing_loss : [0.5600715279579163, 0.9516666531562805]


In [88]:
Y_pred = model.predict(xtest).round(2)

19/19 [==============================] - 0s 1ms/step


In [89]:
Y_pred

array([[0.87, 0.13, 0.  , 0.  ],
       [0.51, 0.48, 0.02, 0.  ],
       [0.29, 0.66, 0.05, 0.  ],
       ...,
       [0.01, 0.39, 0.56, 0.04],
       [0.  , 0.  , 0.05, 0.95],
       [0.  , 0.  , 0.14, 0.86]], dtype=float32)

In [90]:
xtest

array([[-0.4858777 , -1.00142959,  1.33481858, ...,  0.57515083,
        -1.01294087, -0.99714693],
       [-0.55939856,  0.99857245,  0.35275407, ...,  0.57515083,
         0.98722446, -0.99714693],
       [-1.46232663,  0.99857245, -1.24310077, ...,  0.57515083,
         0.98722446,  1.00286124],
       ...,
       [-1.09931738, -1.00142959, -0.87482658, ...,  0.57515083,
         0.98722446,  1.00286124],
       [ 0.48367864, -1.00142959,  0.10723794, ...,  0.57515083,
         0.98722446, -0.99714693],
       [ 0.29757896, -1.00142959,  1.70309277, ...,  0.57515083,
        -1.01294087,  1.00286124]])

In [91]:
Y_pred = [np.argmax(i) for i in Y_pred]
Y_pred

[0,
 0,
 1,
 1,
 3,
 2,
 0,
 2,
 2,
 3,
 0,
 3,
 1,
 1,
 3,
 0,
 0,
 1,
 1,
 1,
 3,
 3,
 1,
 2,
 3,
 2,
 2,
 3,
 2,
 2,
 2,
 1,
 2,
 0,
 3,
 3,
 0,
 0,
 0,
 0,
 2,
 1,
 2,
 1,
 0,
 1,
 2,
 2,
 1,
 2,
 1,
 3,
 1,
 3,
 1,
 3,
 1,
 3,
 3,
 1,
 0,
 2,
 0,
 3,
 2,
 1,
 0,
 2,
 3,
 2,
 1,
 1,
 0,
 3,
 3,
 1,
 2,
 1,
 0,
 0,
 0,
 3,
 1,
 2,
 3,
 2,
 2,
 0,
 1,
 1,
 3,
 0,
 1,
 1,
 2,
 3,
 3,
 0,
 3,
 3,
 3,
 3,
 0,
 1,
 2,
 0,
 0,
 1,
 0,
 2,
 0,
 3,
 1,
 1,
 2,
 2,
 3,
 1,
 2,
 1,
 2,
 0,
 0,
 0,
 3,
 0,
 1,
 1,
 0,
 1,
 0,
 2,
 0,
 3,
 3,
 0,
 3,
 2,
 2,
 1,
 0,
 0,
 3,
 1,
 0,
 2,
 0,
 0,
 0,
 1,
 3,
 3,
 2,
 1,
 0,
 2,
 1,
 0,
 3,
 1,
 3,
 1,
 2,
 2,
 3,
 1,
 2,
 2,
 3,
 2,
 2,
 0,
 3,
 2,
 0,
 3,
 3,
 0,
 0,
 3,
 0,
 3,
 0,
 0,
 2,
 3,
 3,
 1,
 2,
 1,
 2,
 3,
 2,
 2,
 0,
 1,
 2,
 3,
 3,
 1,
 3,
 2,
 0,
 1,
 2,
 2,
 2,
 1,
 0,
 2,
 0,
 3,
 2,
 0,
 2,
 0,
 1,
 0,
 2,
 1,
 0,
 1,
 2,
 1,
 3,
 3,
 0,
 3,
 2,
 1,
 0,
 2,
 3,
 3,
 2,
 1,
 0,
 1,
 3,
 2,
 3,
 2,
 2,
 1,
 0,
 1,
 3,
 0,
 3,
 3,


In [92]:
from sklearn.metrics import classification_report,confusion_matrix

print(classification_report(ytest,Y_pred))
print(confusion_matrix(ytest,Y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       135
           1       0.96      0.93      0.94       149
           2       0.95      0.90      0.93       168
           3       0.94      0.98      0.96       148

    accuracy                           0.95       600
   macro avg       0.95      0.95      0.95       600
weighted avg       0.95      0.95      0.95       600

[[135   0   0   0]
 [  6 138   5   0]
 [  0   6 152  10]
 [  0   0   3 145]]


In [100]:
df_test = pd.read_csv('test.csv')
df_test.head()

,id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,...,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,1,1043,1,1.8,1,14,0,5,0.1,193,...,16,226,1412,3476,12,7,2,0,1,0
1,2,841,1,0.5,1,4,1,61,0.8,191,...,12,746,857,3895,6,0,7,1,0,0
2,3,1807,1,2.8,0,1,0,27,0.9,186,...,4,1270,1366,2396,17,10,10,0,1,1
3,4,1546,0,0.5,1,18,1,25,0.5,96,...,20,295,1752,3893,10,0,7,1,1,0
4,5,1434,0,1.4,0,11,1,49,0.5,108,...,18,749,810,1773,15,8,7,1,0,1


In [101]:
df_test.drop('id',axis=1,inplace=True)

In [102]:
predicted_price = model.predict(df_test)

32/32 [==============================] - 0s 2ms/step


In [103]:
predicted_price

array([[0.        , 0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.        , 0.99999994],
       ...,
       [0.        , 0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.        , 0.99999994],
       [0.        , 0.        , 0.        , 0.99999994]], dtype=float32)

In [97]:
predicted_price = [np.argmax(i) for i in predicted_price]
predicted_price

[3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,


In [98]:
df_test['price_range'] = predicted_price

In [99]:
df_test

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,1043,1,1.8,1,14,0,5,0.1,193,3,...,226,1412,3476,12,7,2,0,1,0,3
1,841,1,0.5,1,4,1,61,0.8,191,5,...,746,857,3895,6,0,7,1,0,0,3
2,1807,1,2.8,0,1,0,27,0.9,186,3,...,1270,1366,2396,17,10,10,0,1,1,3
3,1546,0,0.5,1,18,1,25,0.5,96,8,...,295,1752,3893,10,0,7,1,1,0,3
4,1434,0,1.4,0,11,1,49,0.5,108,6,...,749,810,1773,15,8,7,1,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1700,1,1.9,0,0,1,54,0.5,170,7,...,644,913,2121,14,8,15,1,1,0,3
996,609,0,1.8,1,0,0,13,0.9,186,4,...,1152,1632,1933,8,1,19,0,1,1,3
997,1185,0,1.4,0,1,1,8,0.5,80,1,...,477,825,1223,5,0,14,1,0,0,3
998,1533,1,0.5,1,0,0,50,0.4,171,2,...,38,832,2509,15,11,6,0,1,0,3
